# 24917016 - Zaki Rusydi

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# mengatur perangkat untuk digunakan
# jika GPU tersedia, gunakan GPU, jika tidak, gunakan CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# mengatur hyperparameter dimana total inputan 784 dengan 28x28
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

input_size = 28
sequence_length = 28
hidden_size = 128
num_layers = 2

#### Keterangan

- **num_classes** artinya model mempresiksi class yang berbeda beda sesuai jumlah  yang ditentukn, misalnya 10 class yang berbeda beda
- **num_epochs** artinya model melihat semua data latih sebanyak 2 kali, maksudnya yaitu jika model telah melihat keselurhan data maka iu dihitung 1 epoch
- **batch_size** artinya  data pelatihan akan dibagi berdasarkan jumlah yang ditentukan, misalnya data dibagi dalam satu kelompok dimana berisi 100 gambar.
- **learning_rate** memiliki arti seberapa besar model mengubah bobotnya saat melakukan training model. jika nilai yang ditentukan kecil maka model akan melakukan perubahan secara hati hati, jika menetapkan nilai terlalu besar ditakutkan nantinya model bisa melompat lompat dan menyebabkan model belajar dengan tidak benar.
- **input_size** artinya setiap baris gambar dengan ukuran yang telah ditentukan maka akan dianggap sebagai input. misalnya gambar 28x28
- **sequence_length** berarti model akan membaca langkah tergantung nilai yang ditetapkan. 
- **hidden_size** merupakan jumlah unit dan lapisan tersmbunyi yang ada di dalam model, dimana semakin besar nilainya maka model dapat menyimpan lebih banyak informasi. akan tetapi membuat model menjadi lebih besar dan juga membutuhkan lebih banyak data.
- **num_layers** artinya model menggunakan jumlah layer yang telah ditentukan. misalnya dalam code yaitu model menggunakan 2 layer/lapisan model RNN diamana hal tersebut dapat membantu model agar dapat belajar lebih kompleks lagi.

In [4]:
# melakukan import dataset MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

In [5]:
# mengelompokkan data menjadi batch
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

**Note**: 

saat melakukann pelatihan model data akan diacak setiap epochnya "shuffle=true" berfugsi agar model tidak belajar berdasarkan urutan data. sedangkan pada data test tidak diperlukan pengacakan karena data test hanya berfungsi untuk melakukan test untuk melihat sebarap baik model dapat bekerja.

# RNN

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Menetapkan kondisi awal hidden state
        # x: (n, 28, 28), h0: (2, n, 128)
        # dimana n adalah ukuran batch, 28x28 adalah ukuran gambar, dan 128 adalah hidden size
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Melakukan propagasi maju pada RNN
        # out: (n, 28, 128)
        # output berukuran batch_size x sequence_length x hidden_size
        out, _ = self.rnn(x, h0)
        
        # Mengambil hidden state dari langkah terakhir
        # out: (n, 128)
        # mengambil hasil terakhir dari sequence
        out = out[:, -1, :]
         
        # out: (n, 10)
        # melakukan klasifikasi menggunakan fully connected layer
        out = self.fc(out)
        return out


In [7]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [8]:
# mengukur seberapa jauh hasil prediksi model dari label yang seharusnya
criterion = nn.CrossEntropyLoss()

# menagtur dan melakukanperubahan pada bobot di dalam model berdasarkan loss pada langkah sebelumnya
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [9]:
# Melatih model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Mengubah bentuk gambar dari [N, 1, 28, 28] menjadi [N, 28, 28]
        # agar sesuai dengan input RNN
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Melakukan prediksi dengan model
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Memperbarui bobot model
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Menampilkan hasil pelatihan setiap 100 langkah
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [100/600], Loss: 0.9319
Epoch [1/2], Step [200/600], Loss: 0.8280
Epoch [1/2], Step [300/600], Loss: 0.5276
Epoch [1/2], Step [400/600], Loss: 0.4540
Epoch [1/2], Step [500/600], Loss: 0.4094
Epoch [1/2], Step [600/600], Loss: 0.3768
Epoch [2/2], Step [100/600], Loss: 0.2224
Epoch [2/2], Step [200/600], Loss: 0.3118
Epoch [2/2], Step [300/600], Loss: 0.2541
Epoch [2/2], Step [400/600], Loss: 0.1583
Epoch [2/2], Step [500/600], Loss: 0.1369
Epoch [2/2], Step [600/600], Loss: 0.3224


 **Note:**

 dalam epoch satu loss nilai tinggi dimana awalnya 0.9564 menjadi 0.4793. akan tetapi pada epoch 2 model mengalami kemajuan dimana nilai loss rendah dimulai dari 0.1740 - 0.1965. hal ini memnadakan model belajar dari kesalahan sebelumnya dan semakin baik dalam melakukan proses prediksi

In [10]:
# Test model
# Dalam fase testing,  tidak perlu menghitung gradien (untuk menghemat memori)
with torch.no_grad():

    # menghitung jumlah prediksi yang benar dan menghitung total sampel yang diuji
    n_correct = 0  
    n_samples = 0  

    for images, labels in test_loader:
        # Mengubah bentuk gambar agar sesuai dengan input RNN
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Melakukan prediksi dengan model
        outputs = model(images)
        
        # Mengambil nilai prediksi tertinggi sebagai hasil prediksi
        _, predicted = torch.max(outputs.data, 1)
        
        # Menghitung total sampel dan prediksi yang benar
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    # Menghitung akurasi dalam persen
    acc = 100.0 * n_correct / n_samples
    print(f'Akurasi model pada 10000 gambar test: {acc} %')


Akurasi model pada 10000 gambar test: 92.74 %


# GRU

In [11]:
# menentukan hyperparameter
input_size_gru = 28           
sequence_length_gru = 28      
hidden_size_gru = 128         
num_layers_gru = 2            
num_classes_gru = 10
learning_rate_gru = 0.001     
batch_size_gru = 100          
num_epochs_gru = 10 

In [ ]:
class GRUU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(GRUU, self).__init__()
        
        # batch_first=True berarti input berbentuk (batch_size, sequence_length, input_size)
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        
        # Mengubah output GRU menjadi prediksi kelas
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        
        # # Memproses data melalui GRU, Mengabaikan hidden state terakhir dengan notasi '_', out berukuran (batch_size, sequence_length, hidden_size)
        out, _ = self.gru(x)
        
        # out[:, -1, :] mengambil hidden state terakhir dari setiap sequence
        out = self.fc(out[:, -1, :])
        return out


In [13]:
model_gru = RNN(input_size_gru, hidden_size_gru, num_layers_gru, num_classes_gru).to(device)

In [14]:
# menagtur dan melakukanperubahan pada bobot di dalam model berdasarkan loss pada langkah sebelumnya
optimizer_gru= torch.optim.Adam(model_gru.parameters(), lr=learning_rate)

In [15]:
# Melatih model GRU
n_total_steps_gru = len(train_loader)
for epoch_gru in range(num_epochs_gru):
    for i_gru, (images_gru, labels_gru) in enumerate(train_loader):  
        # Ubah bentuk gambar untuk input GRU: [batch, sequence, features]
        images_gru = images_gru.reshape(-1, sequence_length_gru, input_size_gru).to(device)
        labels_gru = labels_gru.to(device)
        
        # Forward pass
        outputs_gru = model_gru(images_gru)
        loss_gru = criterion(outputs_gru, labels_gru)
        
        # Backward dan optimisasi
        optimizer_gru.zero_grad()
        loss_gru.backward()
        optimizer_gru.step()
        
        # Print progress setiap 100 batch
        if (i_gru+1) % 100 == 0:
            print (f'Epoch [{epoch_gru+1}/{num_epochs_gru}], Step [{i_gru+1}/{n_total_steps_gru}], Loss: {loss_gru.item():.4f}')


Epoch [1/10], Step [100/600], Loss: 1.0016
Epoch [1/10], Step [200/600], Loss: 0.6860
Epoch [1/10], Step [300/600], Loss: 0.5482
Epoch [1/10], Step [400/600], Loss: 0.4782
Epoch [1/10], Step [500/600], Loss: 0.2626
Epoch [1/10], Step [600/600], Loss: 0.4001
Epoch [2/10], Step [100/600], Loss: 0.4304
Epoch [2/10], Step [200/600], Loss: 0.2683
Epoch [2/10], Step [300/600], Loss: 0.3070
Epoch [2/10], Step [400/600], Loss: 0.2253
Epoch [2/10], Step [500/600], Loss: 0.1898
Epoch [2/10], Step [600/600], Loss: 0.1751
Epoch [3/10], Step [100/600], Loss: 0.2438
Epoch [3/10], Step [200/600], Loss: 0.1245
Epoch [3/10], Step [300/600], Loss: 0.0748
Epoch [3/10], Step [400/600], Loss: 0.2596
Epoch [3/10], Step [500/600], Loss: 0.2979
Epoch [3/10], Step [600/600], Loss: 0.1352
Epoch [4/10], Step [100/600], Loss: 0.1193
Epoch [4/10], Step [200/600], Loss: 0.1054
Epoch [4/10], Step [300/600], Loss: 0.1791
Epoch [4/10], Step [400/600], Loss: 0.0672
Epoch [4/10], Step [500/600], Loss: 0.2469
Epoch [4/10

In [16]:
# Testing GRU model
with torch.no_grad():
    n_correct_gru= 0  
    n_samples_gru = 0  
    
    for images_gru, labels_gru in test_loader:
        # Reshape images for GRU input
        images_gru = images_gru.reshape(-1, sequence_length_gru, input_size_gru).to(device)
        labels_gru = labels_gru.to(device)
        
        # Get predictions
        outputs_gru = model_gru(images_gru)
        _, predicted_gru = torch.max(outputs_gru.data, 1)
        
        # Calculate accuracy
        n_samples_gru += labels_gru.size(0)
        n_correct_gru += (predicted_gru == labels_gru).sum().item()

    acc = 100.0 * n_correct_gru / n_samples_gru
    print(f'Accuracy of GRU model on test images: {acc} %')


Accuracy of GRU model on test images: 97.46 %
